### Imports, loading samples, finding dimensions, and printing columns.

In [2]:

import numpy as np 
import pandas as pd 

book_sample = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0', engine = 'pyarrow')
trade_sample = pd.read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0', engine = 'pyarrow')
train_target = pd.read_csv('/kaggle/input/optiver-realized-volatility-prediction/train.csv')

print('book_train sample dimensions:', book_sample.shape)
print('trade_train sample dimensions:', trade_sample.shape)
print('train.csv dimensions:', train_target.shape)

print('book_train columns:', book_sample.columns.tolist())
print('trade_train columns:', trade_sample.columns.tolist())
print('train.csv columns:', train_target.columns.tolist())

book_train sample dimensions: (917553, 10)
trade_train sample dimensions: (123443, 5)
train.csv dimensions: (428932, 3)
book_train columns: ['time_id', 'seconds_in_bucket', 'bid_price1', 'ask_price1', 'bid_price2', 'ask_price2', 'bid_size1', 'ask_size1', 'bid_size2', 'ask_size2']
trade_train columns: ['time_id', 'seconds_in_bucket', 'price', 'size', 'order_count']
train.csv columns: ['stock_id', 'time_id', 'target']


### Using duckdb to get some sql practice and to make data exploration easier
- stock_id: ID number for each of the 112 stocks in the dataset
- time_id: Index for each 10-minute trading window during the day.
- avg_spread: Average (ask price – bid price) across the window (in dollars). Wider spreads mean market makers are unsure = future price swings often larger.
- mid_price_vol: Standard deviation of the mid-price (1/2 (ask+bid)) inside the window. Captures how “jittery” prices already were; choppy windows tend to stay volatile.
- order_imbalance: Net size leaning to buyers vs sellers, Range ≈ –1 to +1. Strong imbalance (big positive or negative) can trigger jumps when the pressure releases.
- trade_logret_std: Std dev of trade-to-trade log-returns within the window. (Micro-volatility seen in executed trades.) Direct glimpse of recent realised volatility. Higher values usually mean the next window will also be lively.

In [36]:
import duckdb
import pyarrow.parquet as pq

sql = """
WITH book AS (
  SELECT
    stock_id,
    time_id,
    AVG(ask_price1 - bid_price1) AS avg_spread,
    STDDEV((ask_price1 + bid_price1) * 0.5) AS mid_price_vol,
    SUM(ask_size1 - bid_size1) * 1.0 /
    NULLIF(SUM(ask_size1 + bid_size1), 0) AS order_imbalance
  FROM read_parquet('/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*/*.parquet')
  GROUP BY stock_id, time_id
),

trade_ret AS (
  SELECT
    stock_id,
    time_id,
    LN(price) - LN(LAG(price) OVER (
        PARTITION BY stock_id, time_id ORDER BY seconds_in_bucket
    )) AS log_ret
  FROM read_parquet('/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/*/*.parquet')
),

trade AS (
  SELECT
    stock_id,
    time_id,
    STDDEV(log_ret) AS trade_logret_std
  FROM trade_ret
  WHERE log_ret IS NOT NULL
  GROUP BY stock_id, time_id
)

/* ---------- FINAL TABLE ---------- */
SELECT
  b.stock_id,
  b.time_id,
  b.avg_spread,
  b.mid_price_vol,
  b.order_imbalance,
  t.trade_logret_std
FROM book AS b
LEFT JOIN trade AS t
USING (stock_id, time_id)
"""

print("Running aggregation …")
features = duckdb.query(sql).to_df()
features.to_parquet("features.parquet")   # cache for later use

print("Done – shape:", features.shape)
features.head()

Running aggregation …


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Done – shape: (428932, 6)


,stock_id,time_id,avg_spread,mid_price_vol,order_imbalance,trade_logret_std
0,0,103,0.001039,0.001901,-0.319490,0.000362
1,0,128,0.000659,0.000921,-0.154892,0.000368
2,0,159,0.000600,0.001411,-0.280426,0.000263
3,0,266,0.000783,0.000894,-0.079256,0.000385
4,0,289,0.000459,0.000584,0.022822,0.000181


### 

- Loaded feature tabel
- Combined featurs with train.csv
- Picked out labels and features

In [37]:
from sklearn.model_selection import GroupKFold

features = pd.read_parquet("features.parquet")      # (≈4 M × 6)

train_target = pd.read_csv(
    "/kaggle/input/optiver-realized-volatility-prediction/train.csv",
    usecols=["stock_id", "time_id", "target"]
)

data = features.merge(train_target, on=["stock_id", "time_id"])
print("Merged shape:", data.shape)
display(data.head())

Merged shape: (428932, 7)


,stock_id,time_id,avg_spread,mid_price_vol,order_imbalance,trade_logret_std,target
0,0,103,0.001039,0.001901,-0.319490,0.000362,0.004120
1,0,128,0.000659,0.000921,-0.154892,0.000368,0.003702
2,0,159,0.000600,0.001411,-0.280426,0.000263,0.002077
3,0,266,0.000783,0.000894,-0.079256,0.000385,0.004280
4,0,289,0.000459,0.000584,0.022822,0.000181,0.003784


- Adds a baseline-volatility feature

In [44]:
data['stock_mean_target'] = (
    data.groupby('stock_id')['target']
        .transform('mean')
)

- Mid-price column: add mid_price = (ask + bid) / 2.
- for each (stock_id, time_id) get first and last mid-price.
- compute % change (mid_price_change).

In [39]:
from glob import glob

book_paths = glob(
    '/kaggle/input/optiver-realized-volatility-prediction/book_train.parquet/*/*.parquet'
)

cols = ['stock_id', 'time_id', 'seconds_in_bucket', 'ask_price1', 'bid_price1']
book_list = [pq.read_table(p, columns=cols).to_pandas() for p in book_paths]
book_raw  = pd.concat(book_list, ignore_index=True)

In [41]:
book_raw['mid_price'] = 0.5 * (book_raw['ask_price1'] + book_raw['bid_price1'])

first_last = (
    book_raw.sort_values(['stock_id', 'time_id', 'seconds_in_bucket'])
            .groupby(['stock_id', 'time_id'])['mid_price']
            .agg(mid_price_first='first', mid_price_last='last')
            .reset_index()
)

first_last['mid_price_change'] = (
    first_last['mid_price_last'] / first_last['mid_price_first'] - 1.0
)

data = data.merge(
    first_last[['stock_id', 'time_id', 'mid_price_change']],
    on=['stock_id', 'time_id'],
    how='left'
)

- list features to use in the model.
- Build matrices: X = features, y = target, groups = time_id for CV.
- Set up time-aware CV (GroupKFold(n_splits=3)).
- Define RMSPE metric.
- Print fold sizes, quick check that each split has balanced train/val rows.

In [45]:
feature_cols = [
    "avg_spread",
    "mid_price_vol",
    "order_imbalance",
    "trade_logret_std",
    'mid_price_change',
    'stock_mean_target' 
]

X = data[feature_cols]
y = data["target"]
groups = data["time_id"]  

gkf = GroupKFold(n_splits=3)

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

for i, (tr_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
    print(f"Fold {i}: train={len(tr_idx):,}  val={len(val_idx):,}")

Fold 0: train=286,028  val=142,904
Fold 1: train=285,918  val=143,014
Fold 2: train=285,918  val=143,014


- Set up LightGBM grid search: Three parameter combos with deeper trees and varied leaf sizes.
- Loop over param sets: For each, perform 3-fold GroupKFold training/validation.
- Early-stopping & logging: Stops after 50 no-gain rounds, prints progress every 200 iters.
- Collect out-of-fold (OOF) predictions
- Track best model: Keeps lowest RMSPE and its parameters, prints final best score.

In [47]:
import lightgbm as lgb 
from lightgbm import early_stopping, log_evaluation

param_grid = [
    {'num_leaves': 127, 'min_data_in_leaf': 50,  'learning_rate': 0.02, 'feature_fraction': 0.8},
    {'num_leaves': 127, 'min_data_in_leaf': 100, 'learning_rate': 0.02, 'feature_fraction': 0.8},
    {'num_leaves': 255, 'min_data_in_leaf': 50,  'learning_rate': 0.015,'feature_fraction': 0.75},
]

def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

best_score = np.inf
best_params = None

for params in param_grid:
    print(f"Testing params: {params}")
    
    oof_pred = np.zeros(len(y))
    
    for fold, (tr_idx, val_idx) in enumerate(gkf.split(X, y, groups)):
        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]
        
        lgb_train = lgb.Dataset(X_tr, y_tr)
        lgb_val   = lgb.Dataset(X_val, y_val, reference=lgb_train)
        
        full_params = {
            'objective': 'regression',
            'metric':    'rmse',
            'verbosity': -1,
            **params
        }
        
        model = lgb.train(
            params = full_params,
            train_set = lgb_train,
            num_boost_round = 5000,
            valid_sets = [lgb_val],
            callbacks = [
                early_stopping(stopping_rounds=50),
                log_evaluation(period=200) 
            ]
        )
        
        oof_pred[val_idx] = model.predict(X_val, num_iteration=model.best_iteration)
        print(f" Fold {fold} done. Best iters: {model.best_iteration}")
    
    score = rmspe(y, oof_pred)
    print(f"OOF RMSPE for params {params}: {score:}")
    
    if score < best_score:
        best_score, best_params = score, params

print("\n==============================")
print(f"Best RMSPE: {best_score:}")
print(f"Best params: {best_params}")

Testing params: {'num_leaves': 127, 'min_data_in_leaf': 50, 'learning_rate': 0.02, 'feature_fraction': 0.8}
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.00150657
[400]	valid_0's rmse: 0.00148638
[600]	valid_0's rmse: 0.00148274
Early stopping, best iteration is:
[728]	valid_0's rmse: 0.00148217
 Fold 0 done. Best iters: 728
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.00159725
[400]	valid_0's rmse: 0.00157153
[600]	valid_0's rmse: 0.00156668
Early stopping, best iteration is:
[711]	valid_0's rmse: 0.00156573
 Fold 1 done. Best iters: 711
Training until validation scores don't improve for 50 rounds
[200]	valid_0's rmse: 0.00144533
[400]	valid_0's rmse: 0.00142825
[600]	valid_0's rmse: 0.00142548
Early stopping, best iteration is:
[717]	valid_0's rmse: 0.00142485
 Fold 2 done. Best iters: 717
OOF RMSPE for params {'num_leaves': 127, 'min_data_in_leaf': 50, 'learning_rate': 0.02, 'feature_fraction': 0.8}: 0.36